#### IberLEF 2022 - DA-VINCIS - Harmful content
#### Violent event identification. 
- Determinate whether a given tweet is associated with a violent incident o not (binary classification)
- No es deteccion de Agresividad, si no si el tweet esta relacionado con un evento de violencia (noticia, ec.)

- https://sites.google.com/view/iberlef2022
- https://codalab.lisn.upsaclay.fr/competitions/2638#learn_the_details-overview

- lematizacion semantica
- lista de issues de clustering con fasttext

- pysentimiento
    - EvoMSA
    - sacar vectores - transformadas a vectores
    
    
- Plan EvoMSA Harmful
    - Fusionar todos los DSs en uno solo directo y entrenar Harmful
        - Haha, Mexa3t2018, Misoginia y MeOffendEs, mas el Harmful
        - y otras ~7 como semeval 
    - Meter los 4 modelos pre-entrenados EvoMSA al EvoMSA de Harmful
    - Variar
        - Clasificadores
            - LinearSVC
            - GaussianNB
            - MultinomialNB
            - TfidfVectorizer
            - MicroTc
                - Tokens List
        - Combinacion de los modelos EvoMSA pre-entrenados
            - Haha, Mexa3t2018, Misoginia y MeOffendEs
    - Comparar clasificadores con 
        - Critical Distance
        - ConfidenceInterval
        - Orange
        - Curva ROC - Recuperar Articulo

In [20]:
pd.set_option('display.max_colwidth', None)

import pandas as pd
train_data = pd.read_csv('train_data.csv', names=['text'], header=None) 
train_labels = pd.read_csv('train_labels_subtask_1.csv', names=['klass'], header=None)

import sklearn.model_selection as model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(train_data, train_labels, train_size=0.8, random_state=True)
#X_train = X_train.squeeze() 
#X_test = X_test.squeeze() 
#y_train = y_train.squeeze() 
#y_test = y_test.squeeze()

X_train.join(y_train).query("klass == 0")

,text,klass
573,"¿Si fueras testigo de un hecho de violencia contra una niña, niño o adolescente sabrías qué hacer? ¿Sabrías cómo denunciar? Te invitamos a participar en este conversatorio el próximo martes 14 de septiembre vía WhatsApp. Únete: https://t.co/eCb0d2i3dt https://t.co/U9CaaHSitO",0
1047,Si hay un accidente de quién sería la culpa....este par de idiotas estuvieron más de 5 minutos allí parados...los carros le pasaban por un lado y ellos inmóviles 🤐#por esto los extraterrestres no nos atacan🤣 https://t.co/7BfibWkRDk,0
774,Aprehenden a dos sujetos por el homicidio de una mujer en Nicolás Romero https://t.co/31umjPGsBF https://t.co/yGOThehN53,0
194,"#UnDíaComoHoy pero de hace 174 años, los #NiñosHéroes resguardaron el Castillo de Chapultepec frente al asalto de tropas invasoras. Honramos su patriotismo, valentía y amor por México.🇲🇽 #DíaDeLosNiñosHéroes https://t.co/vM4P1nnmlq",0
1828,¡Pilas! si te llama algún desconocido a decirte que pertenece a un grupo armado organizado y te exige dinero a cambio de no atentar contra tu vida o la de tus seres queridos te recomendamos llamar a los @GaulaMilitares a la LÍNEA 147. Luchamos contra el secuestro y la extorsión https://t.co/pTYcsqNbXQ,0
...,...,...
2895,"Cuando Riza se encuentra a Barry en el manga ella no llevar a Black Hayate, en el anime si la acampaña Nota:en el manga esto pasa primero, luego lo greed, después la primera aparición de May, más tarde Ed y Al llegan a Rush Valley y el asesinato que Scar hacer pasar mucho después https://t.co/QUPqYQwddJ",0
2763,"#TúOpinasCDN Fueron detenidos esta noche en Puerto Plata otros dos involucrados con la red de narcotráfico y lavado de activos, denominada operación Falcón. https://t.co/Jy4bhXUv5h",0
905,▶ Hijo de Emir Pabón sale del hospital tras nacer prematuro y estar un mes en incubadora El bebé nació justo después del accidente automovilístico por el que sus papás fueron hospitalizados https://t.co/I40UqZ8krj https://t.co/t03N37pZ4Y,0
1096,"#Fiscalía presentó ante un juez de garantías a un hombre que habría abusado sexualmente de su alumna, una menor de 11 años, en #Bogotá. Los hechos investigados ocurrieron entre 2019 y 2021. Información: https://t.co/9G7gZG7lm3 https://t.co/rJt5w7J2TB",0


In [2]:
from EvoMSA.utils import download
from EvoMSA.base import EvoMSA

from sklearn import metrics

from sklearn.metrics import classification_report

haha = download('haha2018_Es.evomsa')
mexa3t = download('mexa3t2018_aggress_Es.evomsa')
misoginia = download('misoginia_Es.evomsa')

In [3]:
evo = EvoMSA(TR=False, B4MSA=True, lang='es',
                 stacked_method='sklearn.naive_bayes.GaussianNB',
                 models=[
                     [misoginia, "sklearn.svm.LinearSVC"],
                     [haha, "sklearn.svm.LinearSVC"] ,
                     [mexa3t, "sklearn.svm.LinearSVC"]
                 ])

import time
start = time.time()
evo.fit(X_train, y_train)    
print(time.time() - start, "seconds")

pred = evo.predict(X_test)
print(pred)

print("F1_Score Macro: ", metrics.f1_score(y_test, pred, average="macro"))

print(classification_report(pred, y_test)) 

C:\Users\Calderas\Anaconda3\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator LinearSVC from version 0.22.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\Calderas\Anaconda3\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator GaussianNB from version 0.22.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
100%|██████████| 3/3 [00:14<00:00,  4.87s/it]

100%|██████████| 1/1 [00:07<00:00,  7.17s/it]

100%|██████████| 4/4 [00:14<00:00,  3.57s/it]

100%|██████████| 5/5 [00:00<00:00,  9.13it/s]


44.44562864303589 seconds


100%|██████████| 3/3 [00:03<00:00,  1.18s/it]

100%|██████████| 1/1 [00:01<00:00,  1.69s/it]

100%|██████████| 4/4 [00:03<00:00,  1.15it/s]

100%|██████████| 4/4 [00:00<00:00, 1318.34it/s]

[0 0 1 ... 1 0 0]
F1_Score Macro:  0.7453061525670066
              precision    recall  f1-score   support

           0       0.88      0.87      0.87       772
           1       0.60      0.64      0.62       240

    accuracy                           0.81      1012
   macro avg       0.74      0.75      0.75      1012
weighted avg       0.82      0.81      0.81      1012

